In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, time
import matplotlib.dates as mdates

In [4]:
#import data
harvest_perrault = pd.read_csv('harvester_perrault.csv')
harvest_pilot = pd.read_csv('harvest_pilot.csv')

how many Perrault Harvester's Operating simultaneously?

In [5]:
harvest_perrault['vpm'] = harvest_perrault['vpm'] * 3 #number of perrault harvesters running per shift

Calculate Productive Time

In [6]:
harvest_perrault['productive_minutes'] = harvest_perrault['strings'] / harvest_perrault['vpm']

In [7]:
perrault = harvest_perrault.groupby('variety')['productive_minutes'].sum().reset_index()

In [8]:
perrault['productive_minutes'].sum()

28872.85114731255

In [9]:
perrault['variety'].unique()

array(['Citra', 'Experimental', 'HBC 1019', 'HBC 522', 'HBC 586',
       'HBC 638', 'HBC 682', 'Idaho 7', 'Mosaic', 'Palisade', 'Sabro',
       'Simcoe', 'Talus'], dtype=object)

In [10]:
perrault['variety'].unique()
# Define the order in which varieties need to be picked
picking_order = [
    'Simcoe', 'Sabro', 'Citra', 'Mosaic', 'HBC 1019', 'Palisade', 'HBC 522', 'Idaho 7', 'Talus', 'HBC 682',
    'HBC 638', 'HBC 586', 'Experimental'
]

# Sort the DataFrame based on the picking order
perrault['order'] = perrault['variety'].apply(lambda x: picking_order.index(x))
perrault = perrault.sort_values(by='order', ascending=True)

In [49]:
from datetime import datetime, timedelta
import pandas as pd

# Function to check if the given date is a working day (Monday to Saturday)
def is_working_day(date):
    return date.weekday() < 6

# Function to find the next working day if the current day is a Sunday
def next_working_day(current_datetime):
    if current_datetime.weekday() == 6:  # Sunday
        current_datetime += timedelta(days=1)
    return current_datetime

# Function to create shift schedule with sequential progression
def create_shift_schedule(data, start_datetime, variety_shift_lengths, break_duration=10, lunch_duration=30):
    schedule = []
    current_datetime = start_datetime

    for index, row in data.iterrows():
        variety = row['variety']
        total_time = row['productive_minutes']
        shift_length = variety_shift_lengths.get(variety, 600)  # Get the shift length for the current variety

        while total_time > 0:
            # Move to the next working day if it's a Sunday
            current_datetime = next_working_day(current_datetime)

            # Determine actual shift time and reduce total time for the variety
            actual_shift_time = min(total_time, shift_length)
            total_time -= actual_shift_time

            schedule.append({
                'variety': variety,
                'ShiftStart': current_datetime,
                'ShiftEnd': current_datetime + timedelta(minutes=actual_shift_time),
                'ShiftTime': actual_shift_time,
                'BreakTime': break_duration,
                'LunchBreak': lunch_duration,
            })

            # Update the current time to the end of this shift
            current_datetime += timedelta(minutes=actual_shift_time)

            # Move to the next working day if shift ends after 04:00 AM
            if current_datetime.time() >= datetime.strptime('04:00', '%H:%M').time():
                current_datetime = datetime.combine(next_working_day(current_datetime), datetime.min.time()) + timedelta(hours=6)

    return pd.DataFrame(schedule)

# Example usage
start_datetime = datetime(2024, 9, 1, 6, 0)

variety_shift_lengths = {
    'Simcoe': 600,  # 10 hours
    'Citra': 600,   # 10 hours
    'Sabro': 600,   # 10 hours
    'Mosaic': 600,  # 10 hours
    'HBC 1019': 450,  # 7.5 hours
    'Palisade': 450,  # 7.5 hours
    'HBC 522': 450,   # 7.5 hours
    'Talus': 450,     # 7.5 hours
    'HBC 682': 450,   # 7.5 hours
    'HBC 638': 450,   # 7.5 hours
    'HBC 586': 450,   # 7.5 hours
    'Experimental': 450  # 7.5 hours
}

# Assuming 'perrault' is your dataframe
shift_schedule = create_shift_schedule(
    data=perrault,
    start_datetime=start_datetime,
    variety_shift_lengths=variety_shift_lengths
)








In [50]:
shift_schedule

,variety,ShiftStart,ShiftEnd,ShiftTime,BreakTime,LunchBreak
0,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00.000000,600.000000,10,30
1,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00.000000,600.000000,10,30
2,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00.000000,600.000000,10,30
3,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00.000000,600.000000,10,30
4,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00.000000,600.000000,10,30
5,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00.000000,600.000000,10,30
6,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00.000000,600.000000,10,30
7,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00.000000,600.000000,10,30
8,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00.000000,600.000000,10,30
9,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00.000000,600.000000,10,30
